Import libraries

In [2]:
import pandas as pd
import numpy as np
from tqdm import trange
import tensorflow as tf

Load the data and compute the user-rating matrix, that has as columns the movies and as rows the ratings of a user

In [3]:
df = pd.read_csv('ratings.csv')

users  = df.user.unique()
movies = df.movie.unique()

user_rating_matrix = np.zeros((len(users), len(movies)))

for i in trange(len(users)):
    for j in range(len(movies)):
        rating = df.loc[df['user'] == users[i]][df['movie'] == movies[j]]['rating'].tolist()

        if rating != []:
            user_rating_matrix[i][j] = int(rating[0]) / 10

100%|██████████| 300/300 [59:52<00:00, 11.97s/it]


Declare and train a autoencoder model

In [77]:
autoencoder = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(user_rating_matrix.shape[1], activation=tf.nn.sigmoid)
])

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(user_rating_matrix, user_rating_matrix, epochs=00, batch_size=20)

Epoch 1/300
15/15 [==============================] - 0s 6ms/step - loss: 0.6676
Epoch 2/300
15/15 [==============================] - 0s 5ms/step - loss: 0.5614
Epoch 3/300
15/15 [==============================] - 0s 4ms/step - loss: 0.4332
Epoch 4/300
15/15 [==============================] - 0s 4ms/step - loss: 0.3779
Epoch 5/300
15/15 [==============================] - 0s 5ms/step - loss: 0.3559
Epoch 6/300
15/15 [==============================] - 0s 5ms/step - loss: 0.3439
Epoch 7/300
15/15 [==============================] - 0s 4ms/step - loss: 0.3364
Epoch 8/300
15/15 [==============================] - 0s 4ms/step - loss: 0.3327
Epoch 9/300
15/15 [==============================] - 0s 5ms/step - loss: 0.3296
Epoch 10/300
15/15 [==============================] - 0s 3ms/step - loss: 0.3272
Epoch 11/300
15/15 [==============================] - 0s 3ms/step - loss: 0.3246
Epoch 12/300
15/15 [==============================] - 0s 3ms/step - loss: 0.3207
Epoch 13/300
15/15 [=================

In [90]:
user_ratings = np.zeros(1132)

print([movies[index] for index in np.argsort(autoencoder.predict(user_ratings.reshape((1, -1))))[0][::-1] if user_ratings[index] == 0][:10])

['https://www.imdb.com/title/tt2305051/', 'https://www.imdb.com/title/tt0108399/', 'https://www.imdb.com/title/tt0105695/', 'https://www.imdb.com/title/tt0317219/', 'https://www.imdb.com/title/tt0111161/', 'https://www.imdb.com/title/tt0084516/', 'https://www.imdb.com/title/tt0120338/', 'https://www.imdb.com/title/tt0829482/', 'https://www.imdb.com/title/tt0119177/', 'https://www.imdb.com/title/tt0110148/']


In [92]:
autoencoder.save('movie_model.h5')